# Loading the python packages

In [9]:
import pandas as pd
from pickle import load
import plotly.graph_objects as go

# Loading the Datasets from Trusted Layer

In [10]:
crypto_ai = pd.read_csv('Trusted/crypto_ai_energy.csv')
world = pd.read_csv('Trusted/world_energy.csv')

# Defining the forecast range

In [11]:
years_ahead = 12
start_year = 2024
years_predict = pd.DataFrame({'Year':range(start_year,start_year+years_ahead,1)})

# Loading the Models from Models folder

In [12]:
# load the model from disk
crypto_ai_model = load(open('Models/crypto_ai_forecast.sav', 'rb'))
world_model = load(open('Models/world_energy_forecast.sav', 'rb'))
#rescaledValidationX = scaler.transform(X_validation) #in case the data is scaled.
#predictions = model.predict(rescaledValidationX)
predictions_cd = crypto_ai_model.get_forecast(steps = years_ahead, exog = years_predict)
predictions_w = world_model.get_forecast(steps = years_ahead, exog = years_predict)

# Crypto and AI Energy Consumption Forecast

In [13]:
forecast_cd = pd.concat([years_predict, predictions_cd.predicted_mean.reset_index(drop='True'), predictions_cd.conf_int().reset_index(drop='True')], axis=1)
forecast_cd.head()

,Year,predicted_mean,lower Total Consumption,upper Total Consumption
0,2024,660.443642,597.258330,723.628954
1,2025,743.903376,626.296452,861.510300
2,2026,824.533954,636.364386,1012.703522
3,2027,906.381480,639.286231,1173.476728
4,2028,987.705541,632.349114,1343.061968


In [14]:
# Create Plotly figure
fig = go.Figure()

# Observed data
fig.add_trace(go.Scatter(
    x=crypto_ai['Year'], y=crypto_ai['Total Consumption'],
    mode='lines', name='Observed',
    line=dict(color='black')
))

# Mean forecast
fig.add_trace(go.Scatter(
    x=forecast_cd['Year'], y=forecast_cd['predicted_mean'],
    mode='lines', name='Forecast',
    line=dict(color='blue')
))

# Confidence interval (as shaded area)
fig.add_trace(go.Scatter(
    x=forecast_cd['Year'].tolist() + forecast_cd['Year'].tolist()[::-1], # x for upper and then lower reversed
    y=forecast_cd['lower Total Consumption'].tolist() + forecast_cd['upper Total Consumption'].tolist()[::-1],
    fill='toself',
    fillcolor='rgba(0, 0, 255, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    name='95% CI'
))

fig.update_layout(
    title="Crypto and AI Energy Consumption - ARIMA Forecast with Confidence Interval",
    xaxis_title="Date",
    yaxis_title="Value",
    template="plotly_white"
)

fig.show()

# World Energy Consumption Forecast

In [15]:
forecast_w = pd.concat([years_predict, predictions_w.predicted_mean.reset_index(drop='True'), predictions_w.conf_int().reset_index(drop='True')], axis=1)
forecast_w.head()

,Year,predicted_mean,lower Consumption,upper Consumption
0,2024,118530.244519,115367.113083,121693.375955
1,2025,119852.830079,115337.823770,124367.836387
2,2026,121175.415638,115595.124875,126755.706402
3,2027,122498.001198,115996.579032,128999.423364
4,2028,123820.586758,116487.659782,131153.513733


In [16]:
# Create Plotly figure
fig = go.Figure()

# Observed data
fig.add_trace(go.Scatter(
    x=world['Year'], y=world['Consumption'],
    mode='lines', name='Observed',
    line=dict(color='black')
))

# Mean forecast
fig.add_trace(go.Scatter(
    x=forecast_w['Year'], y=forecast_w['predicted_mean'],
    mode='lines', name='Forecast',
    line=dict(color='blue')
))

# Confidence interval (as shaded area)
fig.add_trace(go.Scatter(
    x=forecast_w['Year'].tolist() + forecast_w['Year'].tolist()[::-1], # x for upper and then lower reversed
    y=forecast_w['lower Consumption'].tolist() + forecast_w['upper Consumption'].tolist()[::-1],
    fill='toself',
    fillcolor='rgba(0, 0, 255, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    name='95% CI'
))

fig.update_layout(
    title="Total World Energy Consumption - ARIMA Forecast with Confidence Interval",
    xaxis_title="Date",
    yaxis_title="Value",
    template="plotly_white"
)

fig.show()